## Build U-NET++ semantic segmentation model using PyTorch 🔥

### Datasets and DataLoaders

In [21]:
import glob
import rasterio
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.utils as smp_utils
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from common.constants import NODATA_FLOAT32


import numpy as np

torch.manual_seed(666)

def get_chip(path):
    return path.split('/')[-1].replace('.tif', '')


class LandcoverDataset(Dataset):
    def __init__(self, root_dir, sub_dir):
        self.data_dir = f'{root_dir}/{sub_dir}'
                        
        image_paths = glob.glob(f'{self.data_dir}/source/*.tif')        
        label_paths = glob.glob(f'{self.data_dir}/labels/*.tif')
        
        image_paths = sorted(image_paths, key=lambda x: x.split('/')[-1])
        label_paths = sorted(label_paths, key=lambda x: x.split('/')[-1])
        
        assert len(image_paths) == len(label_paths)

        self.image_paths = image_paths
        self.label_paths = label_paths
        
        print(f'{sub_dir}... {len(self.image_paths)} chips')
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
                
        image_path = self.image_paths[index]
        label_path = self.label_paths[index]
        
        image_name = image_path.split('/')[-1]
        label_name = label_path.split('/')[-1]        
        assert image_name == label_name
        
        with rasterio.open(image_path) as src:
            image = src.read(masked=True)
            image = image.filled(-1.0) # fill with -1.0 rather than -9999.0
            image = torch.from_numpy(image)
        with rasterio.open(label_path) as src:
            label = src.read(1, masked=True)
            label = label.filled(0)
            label = torch.from_numpy(label).long()     
        
        return image, label

landcover_training_dir = "./data/mergedLandcoverTrainingData/split"

train_dataset = LandcoverDataset(root_dir=landcover_training_dir, sub_dir='train')
train_dataloader = DataLoader(train_dataset, batch_size=8, num_workers=0, shuffle=True)

val_dataset = LandcoverDataset(root_dir=landcover_training_dir, sub_dir='val')
val_dataloader = DataLoader(val_dataset, batch_size=8, num_workers=0, shuffle=True)

test_dataset = LandcoverDataset(root_dir=landcover_training_dir, sub_dir='test')
test_dataloader = DataLoader(test_dataset, batch_size=8, num_workers=0, shuffle=True)

train... 294 chips
val... 36 chips
test... 38 chips


In [17]:
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.utils as smp_utils
import torch.optim as optim


# make sure this is in sync with create patches
LANDCOVER_CLASSES = {
    'unclassified': 0,
    'cloud' : 1,
    'bare_ground' : 2,
    'built' : 3,
    'trees' : 4,
    'burned' : 5,
    'semi_natural_vegetation' : 6,
    'agriculture' : 7,
    'water' : 8,
}

ACTIVATION = None
ENCODER = 'resnet18' 
LOSS = 'dice'

model = smp.UnetPlusPlus(
    encoder_name=ENCODER,
    encoder_depth=5, 
    encoder_weights='imagenet', 
    decoder_use_batchnorm=True, 
    decoder_channels=(256, 128, 64, 32, 16), 
    decoder_attention_type=None,
    in_channels=4, 
    classes=len(LANDCOVER_CLASSES), 
    activation=ACTIVATION
)

if LOSS == 'dice':
    loss = smp.losses.DiceLoss(mode="multiclass")
    loss.__name__ = 'Dice_loss'
else:
    raise ValueError("loss not implemented")

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)


In [9]:
import datetime

now = datetime.datetime.now()
date_string = now.strftime("%Y%m%d")
model_path =  f'./landcover_classification_model_{ENCODER}_{LOSS}_{date_string}.pth'
print(model_path)


./landcover_classification_model_resnet18_dice_20230328.pth


In [22]:

print(model_path)

metrics = []

train_epoch = smp_utils.train.TrainEpoch(
    model, 
    loss=loss,
    metrics= metrics,
    optimizer=optimizer,
    verbose=True,
)

val_epoch = smp_utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    verbose=True,
)


# 2 epochs, 294 training chips: 0.4256 dice loss

NUM_EPOCHS = 4

for i in range(0, NUM_EPOCHS):
    train_logs = train_epoch.run(train_dataloader)
    val_logs = val_epoch.run(val_dataloader)
    torch.save(model, model_path)
    
print('done')

./landcover_classification_model_resnet18_dice_20230328.pth
valid: 100%|██████████| 5/5 [00:13<00:00,  2.63s/it, Dice_loss - 0.3734]
done


In [23]:

test_epoch = smp_utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    verbose=True,
)

test_logs = test_epoch.run(test_dataloader)


valid: 100%|██████████| 5/5 [00:15<00:00,  3.09s/it, Dice_loss - 0.4256]


In [39]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from common.utilities.visualization import plot_bands


count = 3
offset = 13


fig, rows = plt.subplots(count, 3, figsize=(16, 16))
if rows.ndim == 1:
    rows = np.expand_dims(rows, 0)
    
    
best_model = torch.load(model_path)


for i in range(count):
            
    row = rows[i]    
    image, labels = test_dataset[i+offset]       
        
    image = np.expand_dims(image, 0)     
    image = torch.tensor(image)
    
    prediction = best_model.predict(image)
    probabilities = torch.sigmoid(prediction)
    
    prediction = torch.argmax(probabilities, dim=1)

    prediction = (prediction.squeeze().cpu().numpy().round())
    prediction = np.ma.array(prediction, mask=(prediction==0))

    labels = labels.squeeze().numpy()
    labels = np.ma.array(labels, mask=(labels==0))
    
    print('labels', np.unique(labels))
    print('prediction', np.unique(prediction))
    
    image = image.cpu().numpy().squeeze(0)
    plot_bands(image, bands=[2, 1, 0], ax=row[0], transpose=True)
    row[0].set_title('Image')
    
    row[1].imshow(labels, vmin=0, vmax=13)
    row[1].set_title('Truth')
    
    row[2].imshow(prediction, vmin=0, vmax=13)
    row[2].set_title('Prediction')
    
    

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


labels [2 4 6 8 --]
prediction [1 2 3 4 5 6 7 8 --]
labels [4 6 8 --]
prediction [1 2 3 4 5 6 7 8 --]
labels [4 6 8 --]
prediction [1 2 3 4 5 6 7 8 --]
